# Clinical Trial Prediction Dataset Generation

This notebook generates a binary forecasting dataset of clinical trial outcomes using Lightning Rod's Future-as-Label methodology.

## Overview

We'll use the Lightning Rod SDK to:
1. Pull historical news articles about clinical trials (2023-2024)
2. Generate forward-looking prediction questions
3. Automatically label outcomes using verified results from late 2024/2025
4. Create train/test splits for model training

## What You'll Generate

- **~1,800 total questions** from clinical trial news
- **~1,400 high-confidence labeled examples** (72-75% validity rate)
- **Training set** (85% of data)
- **Test set** (15% of data)

## Requirements

- Lightning Rod API key ([get free credits](https://www.lightningrod.ai/))
- Python 3.10+
- **Can run on local laptop** (no GPU needed)

## Time Required

~2-4 minutes total

## Output Files

- `clinical_data.csv` - Full valid dataset
- `clinical_train.csv` - Training set (85%)
- `clinical_test.csv` - Test set (15%)

---

Let's get started! 🚀

# Install Lightning Rod

In [5]:
%pip install lightningrod-ai python-dotenv

from IPython.display import clear_output
clear_output()

## Set up the client

Sign up at [dashboard.lightningrod.ai](https://dashboard.lightningrod.ai/?redirect=/api) to get your API key and **$50 of free credits**.

- **Google Colab**: Go to the Secrets section (key icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`
- **Local Jupyter**: Set the `LIGHTNINGROD_API_KEY` environment variable, or you'll be prompted to enter it

In [6]:
from dotenv import load_dotenv
from lightningrod import LightningRod
from lightningrod.utils import config

load_dotenv()
api_key = config.get_config_value("LIGHTNINGROD_API_KEY")

lr = LightningRod(api_key=api_key)

## Binary Answer Type Overview

Binary questions have two possible answers: Yes/No or True/False. They're ideal for:
- Event prediction ("Will X happen?")
- Outcome forecasting ("Will Y succeed?")
- Decision-making scenarios ("Should we do Z?")

Binary questions work with any data source: News Search, Top Aggregated News, or Custom Documents.

In [ ]:
from datetime import datetime
from lightningrod import QuestionGenerator, WebSearchLabeler, QuestionRenderer, QuestionPipeline, BinaryAnswerType, NewsSeedGenerator

answer_type = BinaryAnswerType()

# Seed generator for clinical trial news from 2022-2024
seed_generator = NewsSeedGenerator(
    start_date=datetime(2022, 1, 1),
    end_date=datetime(2024, 12, 31),
    search_query=[
        "clinical trial Phase 3 results",
        "FDA drug approval",
        "pharmaceutical trial outcomes",
        "biotech Phase 2 trial",
        "clinical study endpoints",
        "FDA approves",
        "trial meets endpoints",
        "drug approval decision",
    ],
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate binary forecasting questions about clinical trial outcomes. "
        "Questions should focus on whether trials will meet their primary endpoints, "
        "receive FDA approval, complete on schedule, or achieve specific clinical milestones. "
        "Questions must be specific with company names, drug names, trial phases, and target dates. "
        "All questions should be answerable with Yes or No based on publicly available trial results."
    ),
    examples=[
        "Will Novo Nordisk's CagriSema Phase 3 trial meet its primary endpoints by December 31, 2024?",
        "Will the FDA approve Eli Lilly's donanemab for Alzheimer's disease by June 30, 2024?",
        "Will Moderna's RSV vaccine complete Phase 3 trials by September 2024?",
        "Will Pfizer's obesity drug danuglipron advance to Phase 3 by March 2024?",
        "Will Vertex Pharmaceuticals' pain medication VX-548 receive FDA approval by Q4 2024?",
    ],
    bad_examples=[
        "What will be the side effects of the new drug?",  # Not binary
        "How many patients will enroll in the trial?",  # Not about outcomes
        "When will the company announce results?",  # Too vague
        "Will the drug be successful?",  # Not specific enough (no timeline, no drug name)
        "Is the trial still ongoing?",  # Not forward-looking
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
# This will search for actual trial results and FDA announcements that occurred after the question date
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

# Renderer formats the question output
renderer = QuestionRenderer(answer_type=answer_type)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

## Run the Pipeline

Generate binary questions

In [14]:
dataset = lr.transforms.run(pipeline_config, max_questions=2000, name="Binary answer type`") # keep max questions low when testing

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  >> Pipeline Running                                                                                            │
│                                                                                                                 │
│    Total cost: $68.84                                                                                           │
│                                                                                                                 │
│  ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓  │
│  ┃ Step                           ┃ Progress                ┃    In ┃   Out ┃  Rejected ┃  Errors ┃ Duration ┃  │
│  ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩  │
│  │ NewsSeedGeneratorTransform     │ Complete                │   200 │  1982 │         0 │       0 │      40s │  │
│  │ QuestionGeneratorTransform     │ In progress             │  1982 │  1890 │        91 │       0 │      13s │  │
│  │ WebSearchLabelerTransform      │ In progress             │  1890 │  1439 │       450 │       0 │      20s │  │
│  │ QuestionRendererTransform      │ In progress             │  1439 │  1439 │         0 │       0 │       0s │  │
│  └────────────────────────────────┴─────────────────────────┴───────┴───────┴───────────┴─────────┴──────────┘  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [16]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [25]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
rows = dataset.flattened()
df = pd.DataFrame(rows)

print(f"Generated {dataset.num_rows} samples (%.1f%% valid)\n" % (dataset.valid_count() / dataset.num_rows * 100))

df[["question.question_text", "label.label", "label.label_confidence", "is_valid", "meta.filter_reason"]].rename(
    columns={
        "question.question_text": "Question",
        "label.label": "Answer",
        "label.label_confidence": "Confidence",
        "is_valid": "Valid",
        "meta.filter_reason": "Invalid Reason"
    }
)

Generated 1882 samples (72.6% valid)



,Question,Answer,Confidence,Valid,Invalid Reason
0,Will the InTandem neurorehabilitation system (...,1,1.0,True,NaN
1,Will Eli Lilly's SURMOUNT-2 clinical trial for...,1,1.0,True,NaN
2,Will the FDA grant full approval to a targeted...,1,1.0,True,NaN
3,Will Abivax's lead candidate Phase 3 clinical ...,1,1.0,True,NaN
4,Will Rapa Therapeutics begin its NIH-funded Ph...,0,1.0,True,NaN
...,...,...,...,...,...
1877,Will Novo Nordisk's Phase 3 REDEFINE-1 trial f...,1,1.0,True,NaN
1878,Did the FDA officially delay the enforcement o...,1,1.0,True,NaN
1879,Will ARS Pharmaceuticals' epinephrine nasal sp...,1,1.0,False,Resolution date is before seed creation date
1880,Will Eli Lilly's Phase 3 TRAILBLAZER-ALZ 2 tri...,1,1.0,True,NaN


In [27]:
new_df = df[["question.question_text", "label.label", "label.label_confidence", "is_valid", "meta.filter_reason"]].rename(
    columns={
        "question.question_text": "Question",
        "label.label": "Answer",
        "label.label_confidence": "Confidence",
        "is_valid": "Valid",
        "meta.filter_reason": "Invalid Reason"
    }
)

# Keep only valid samples and save full dataset

In [ ]:
df_valid = new_df[new_df["Valid"] == True]
final_df = df_valid.drop(columns=["Invalid Reason"])

In [39]:
csv_filename_full = "clinical_data.csv"
final_df.to_csv(csv_filename_full, index=False)
print(f"\n✅ Full valid dataset saved to: {csv_filename_full}")


✅ Full valid dataset saved to: clinical_data.csv


# Split into train (85%) and test (15%)

In [41]:
from sklearn.model_selection import train_test_split

print(f"Total samples: {len(final_df)}")
print(f"\nAnswer distribution:")
print(final_df['Answer'].value_counts())

# Split into train (85%) and test (15%)
train_df, test_df = train_test_split(
    final_df,
    test_size=0.15,
    random_state=42,
    stratify=final_df['Answer']  # Ensures balanced classes in both sets
)

print(f"\nTrain set: {len(train_df)} samples")
print(f"Test set: {len(test_df)} samples")

print(f"\nTrain Answer distribution:")
print(train_df['Answer'].value_counts())
print(f"\nTest Answer distribution:")
print(test_df['Answer'].value_counts())

# Save the splits
train_df.to_csv("clinical_train.csv", index=False)
test_df.to_csv("clinical_test.csv", index=False)

print("\n✅ Data split complete!")
print("📁 Saved: clinical_train.csv, clinical_test.csv")


Total samples: 1367

Answer distribution:
Answer
1    748
0    619
Name: count, dtype: int64

Train set: 1161 samples
Test set: 206 samples

Train Answer distribution:
Answer
1    635
0    526
Name: count, dtype: int64

Test Answer distribution:
Answer
1    113
0     93
Name: count, dtype: int64

✅ Data split complete!
📁 Saved: clinical_train.csv, clinical_test.csv
